<a href="https://colab.research.google.com/github/PochkaChaiki/gasReserves/blob/main/GasReservesCalculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [137]:
from dataclasses import dataclass, asdict
import pandas as pd
import numpy as np

Constants

In [138]:
zero_c_to_k = 273
norm_temp_c = 20

## Initial data

In [139]:
@dataclass
class Data:
    value: float
    min: float
    max: float

@dataclass
class StatData:
    mean: float
    sigma: float

In [140]:
df_init_data = pd.DataFrame(columns=['area', 'effective_thickness', 'porosity_coef', 'gas_saturation_coef', 'initial_reservoir_pressure',
                                     'relative_density', 'reservoir_temp', 'permeability'],
                            index=['value', 'min', 'max'])

df_init_data['area']                       = {"value": 38_556, "min":36_628.20, "max":40_483.80}
df_init_data['effective_thickness']        = {"value":11.10,   "min":10.50,     "max":12.15}
df_init_data['porosity_coef']              = {"value":0.09,    "min":0.01,      "max":0.13}
df_init_data['gas_saturation_coef']        = {"value":0.7,     "min":0.65,      "max":0.75}
df_init_data['initial_reservoir_pressure'] = {"value":32.30,   "min":29.07,     "max":35.53}
df_init_data['relative_density']           = {"value":0.63,    "min":0.63,      "max":0.64}
df_init_data['reservoir_temp']             = {"value":320.49,  "min":288.44,    "max":352.54}
df_init_data['permeability']               = {"value":0.75,    "min":0,         "max":19.14}

In [141]:
df_init_data

,area,effective_thickness,porosity_coef,gas_saturation_coef,initial_reservoir_pressure,relative_density,reservoir_temp,permeability
value,38556.0,11.10,0.09,0.70,32.30,0.63,320.49,0.75
min,36628.2,10.50,0.01,0.65,29.07,0.63,288.44,0.00
max,40483.8,12.15,0.13,0.75,35.53,0.64,352.54,19.14


## Results

In [142]:
df_result_data = pd.DataFrame(columns=['area_volume', 'pore_volume', 'temp_correction', 'fin_reservoir_pressure',
                                       'critical_pressure', 'critical_temp', 'init_overcompress_coef',
                                       'fin_overcompress_coef', 'geo_gas_reserves', 'dry_gas_init_reserves',
                                       'relative_density'],
                              index=['value', 'min', 'max'])

df_result_data['area_volume']            = df_init_data['area'] * df_init_data['effective_thickness']
df_result_data['pore_volume']            = df_result_data['area_volume'] * df_init_data['porosity_coef']
df_result_data['temp_correction']        = (zero_c_to_k * 2 + norm_temp_c) / (zero_c_to_k + df_init_data['reservoir_temp'] )
df_result_data['fin_reservoir_pressure'] = np.exp(1293 * 1e-9 * 2700 * df_init_data['relative_density']) # What are this magical numbers?
df_result_data['critical_pressure']      = 4.892 - 0.4048 * df_init_data['relative_density'] # Same question
df_result_data['critical_temp']          = 94.717 + 170.8 * df_init_data['relative_density'] # Same question
df_result_data['init_overcompress_coef'] = (0.4 * np.log10(df_init_data['reservoir_temp'] / df_result_data['critical_temp']) + 0.73)**(df_init_data['initial_reservoir_pressure'] / df_result_data['critical_pressure']) + 0.1 * df_init_data['initial_reservoir_pressure'] / df_result_data['critical_pressure'] # Same question
df_result_data['fin_overcompress_coef']  = (0.4 * np.log10(df_init_data['reservoir_temp'] / df_result_data['critical_temp']) + 0.73)**(df_result_data['fin_reservoir_pressure'] / df_result_data['critical_pressure']) + 0.1 * df_result_data['fin_reservoir_pressure'] / df_result_data['critical_pressure'] # Same question
df_result_data['geo_gas_reserves']       = df_init_data['gas_saturation_coef'] * df_init_data['initial_reservoir_pressure'] * df_result_data['pore_volume'] * df_result_data['temp_correction'] * df_result_data['init_overcompress_coef']
df_result_data['dry_gas_init_reserves']  = df_result_data['geo_gas_reserves'] * (100 - 0.012 - 0.003 - 0.012) / 100 # You're kidding me! Same question
df_result_data

,area_volume,pore_volume,temp_correction,fin_reservoir_pressure,critical_pressure,critical_temp,init_overcompress_coef,fin_overcompress_coef,geo_gas_reserves,dry_gas_init_reserves,relative_density
value,427971.60,38517.4440,0.953681,1.002202,4.636976,202.321,0.926820,0.977068,7.697622e+05,7.695544e+05,NaN
min,384596.10,3845.9610,1.008122,1.002202,4.636976,202.321,0.857983,0.972359,6.285718e+04,6.284021e+04,NaN
max,491878.17,63944.1621,0.904818,1.002237,4.632928,204.029,0.995625,0.980873,1.535022e+06,1.534607e+06,NaN
